In [121]:
import dataclasses
from grafana_api.grafana_face import GrafanaFace
from typing import Dict, Optional, List

In [122]:
GRAFANA_API = GrafanaFace(auth=("admin","admin"), host='localhost:3000')

In [123]:
members = dict()    # members[team_name]    -> team_id
teams = dict()      # teams[team_name]      -> team_id

In [124]:
# @dataclasses.dataclass(frozen=True)
class User(Dict):
    name: str
    email: str
    password: str
    login: Optional[str] = None
    orgId: Optional[int] = None

class Team(Dict):
    name: str
    email: str
    orgId: Optional[int] = None


In [125]:
def make_member(grafana_api:GrafanaFace, user:User):
    return grafana_api.admin.create_user(user)

def make_team(grafana_api:GrafanaFace, team:Team):
    response = grafana_api.teams.add_team(team)
    if response["message"] == "Team created":
        return response["teamId"]
    return -1

def add_member_to_team(grafana_api:GrafanaFace, user_id:str, team_id:int):
    return grafana_api.teams.add_team_member(team_id, user_id)

def get_team_members_by_team_id(grafana_api:GrafanaFace, team_id:int):
    return grafana_api.teams.get_team_members(team_id)


In [126]:
def get_n_mock_teams(n) -> List[Team]:
    return [Team({"name": f"team{i}", "email": f"team{i}@gmail.com"}) for i in range(n)]

def get_n_mock_users(n) -> List[User]:
    return [User({"name":f"bruker{i}", "email":f"bruker{i}@gmail.com", "password":"password"}) for i in range(n)]

In [127]:
mock_users = get_n_mock_users(3)

for user in mock_users:
    member = make_member(GRAFANA_API, user)
    members[user["email"]] = member['id']

In [128]:
mock_teams = get_n_mock_teams(4)

for team in mock_teams:
    team_id = make_team(GRAFANA_API, team)
    teams[team['name']] = team_id